In [4]:
import scanpy as sc
import pandas as pd
import random
import numpy as np
import matplotlib as plt
import gc 
import ctypes
import scvi

/home/biolab/miniconda3/lib/python3.13/site-packages/scvi/__init__.py:31: DeprecationWarning: scvi is deprecated, please uninstall scvi via `pip uninstall scvi` and install the new scvi-tools package at github.com/YosefLab/scvi-tools
  warnings.warn(deprecation_msg, DeprecationWarning)


In [6]:
adata = sc.read_h5ad('../data/luca_query_reannotated.h5ad')

In [7]:
adata.obs

,sample,uicc_stage,ever_smoker,age,donor_id,origin,dataset,ann_fine,cell_type_predicted,doublet_status,...,IFN_TAMs,Reg_TAMs,Inflam_TAMs,LA_TAMs,Angio_TAMs,RTM_TAMs,Prolif_TAMs,Subtype,Projection_CellType,ident
001C_AAACCTGTCAACACCA-0,Adams_Kaminski_2020_001C,non-cancer,no,22.0,Adams_Kaminski_2020_001C,normal,Adams_Kaminski_2020,Macrophage alveolar,Macrophage,singlet,...,0.353933,0.571664,0.298638,0.541069,0.427279,0.565822,0.401695,Reg_TAMs,RTM_TAMs,local
001C_AAACGGGAGGCTCATT-0,Adams_Kaminski_2020_001C,non-cancer,no,22.0,Adams_Kaminski_2020_001C,normal,Adams_Kaminski_2020,Macrophage,Macrophage,singlet,...,0.477315,0.451494,0.370148,0.492239,0.537751,0.511095,0.348617,Angio_TAMs,LA_TAMs,local
001C_AAACGGGGTATAATGG-0,Adams_Kaminski_2020_001C,non-cancer,no,22.0,Adams_Kaminski_2020_001C,normal,Adams_Kaminski_2020,Macrophage alveolar,Macrophage,singlet,...,0.387389,0.497444,0.282365,0.403790,0.443018,0.548354,0.328568,RTM_TAMs,RTM_TAMs,local
001C_AAACGGGTCACAAACC-0,Adams_Kaminski_2020_001C,non-cancer,no,22.0,Adams_Kaminski_2020_001C,normal,Adams_Kaminski_2020,Macrophage,Macrophage,singlet,...,0.333177,0.592321,0.262999,0.594631,0.444907,0.450925,0.406192,LA_TAMs,LA_TAMs,local
001C_AAAGATGAGTGCTGCC-0,Adams_Kaminski_2020_001C,non-cancer,no,22.0,Adams_Kaminski_2020_001C,normal,Adams_Kaminski_2020,Macrophage alveolar,Macrophage,singlet,...,0.358754,0.648105,0.282604,0.482257,0.408499,0.454457,0.291259,Reg_TAMs,RTM_TAMs,local
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCAAGCTGTCTA-1-38-8,Leader_Merad_2021_414,II,no,64.0,Leader_Merad_2021_729,tumor_primary,Leader_Merad_2021_10x_3p_v2_beads,Macrophage,NaN,singlet,...,0.551482,0.640531,0.489101,0.623875,0.525708,0.514441,0.527691,Reg_TAMs,IFN_TAMs,local
TTTGTCAAGGATATAC-1-38-8,Leader_Merad_2021_414,II,no,64.0,Leader_Merad_2021_729,tumor_primary,Leader_Merad_2021_10x_3p_v2_beads,Macrophage,NaN,singlet,...,0.492014,0.699073,0.539889,0.612688,0.608443,0.551273,0.460248,Reg_TAMs,Reg_TAMs,local
TTTGTCAAGTACGCGA-1-38-8,Leader_Merad_2021_414,II,no,64.0,Leader_Merad_2021_729,tumor_primary,Leader_Merad_2021_10x_3p_v2_beads,Macrophage,NaN,singlet,...,0.470449,0.588994,0.611164,0.427515,0.471975,0.431026,0.455491,Inflam_TAMs,Inflam_TAMs,local
TTTGTCACATCTATGG-1-38-8,Leader_Merad_2021_414,II,no,64.0,Leader_Merad_2021_729,tumor_primary,Leader_Merad_2021_10x_3p_v2_beads,Macrophage,NaN,singlet,...,0.515349,0.589936,0.633147,0.577088,0.550079,0.487074,0.425062,Inflam_TAMs,LA_TAMs,local


In [8]:
adata.obs['Projection_CellType'].value_counts()


Projection_CellType
RTM_TAMs       94074
LA_TAMs        32428
Reg_TAMs       17933
IFN_TAMs       12215
Inflam_TAMs     6484
Angio_TAMs      5741
Int.Node.3      4221
Prolif_TAMs     3771
Int.Node.4       922
Int.Node.5       150
Name: count, dtype: int64

In [9]:
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats

In [10]:
import gc 
import ctypes

In [6]:
def make_pseudobulk(adata, sample_colname, cell_type_colname, metacols):
    '''
    To create an an data with pseudo-bulk gene expression profiles for the samples and cell_types given.
    The function also carries metadata columns from the obs. metacols: as a list of col names.
    This is how to use:
        ps_adata = make_pseudobulk(hlca, sample_colname = 'sample', cell_type_colname = 'cell_type', metacols= ['patient', 'disease', 'tissue', 'study', 'sex'])
    '''
    ps_list = []
    for s in adata.obs[sample_colname].unique():
        adata_sub = adata[adata.obs[sample_colname] == s]
        
        for ct in adata_sub.obs[cell_type_colname].unique():
            adata_sub_ct = adata_sub[adata_sub.obs[cell_type_colname] == ct]
            
            adata_rep = sc.AnnData(X = adata_sub_ct.X.sum(axis = 0),
                                   var = adata_sub_ct.var[[]])
            samp_name=adata_sub.obs[sample_colname].iloc[0]
            cell= adata_sub_ct.obs[cell_type_colname].iloc[0]
            adata_rep.obs_names = [f"{samp_name}{cell}"]
            adata_rep.obs[cell_type_colname] = adata_sub_ct.obs[cell_type_colname].iloc[0]
            adata_rep.obs[metacols] = adata_sub_ct.obs[metacols].iloc[0]
            ps_list.append(adata_rep)

    ps_adata = sc.concat(ps_list)
    
    return ps_adata


In [ ]:
def make_pseudobulk2(adata, sample_colname, cell_type_colname, metacols):
    '''
    Creates an AnnData object with pseudo-bulk gene expression profiles 
    for the samples and cell types specified. Also adds the number of cells 
    used per pseudobulk as "n_cells" in .obs.

    Parameters:
        adata: AnnData
        sample_colname: str
        cell_type_colname: str
        metacols: list of str
    
    Returns:
        AnnData with pseudobulked profiles and metadata.
    '''
    ps_list = []
    for s in adata.obs[sample_colname].unique():
        adata_sub = adata[adata.obs[sample_colname] == s]
        
        for ct in adata_sub.obs[cell_type_colname].unique():
            adata_sub_ct = adata_sub[adata_sub.obs[cell_type_colname] == ct]
            
            # Count how many cells are in this group
            n_cells = adata_sub_ct.shape[0]
            
            adata_rep = sc.AnnData(X=adata_sub_ct.X.sum(axis=0),
                                   var=adata_sub_ct.var[[]])
            
            samp_name = adata_sub.obs[sample_colname].iloc[0]
            cell = adata_sub_ct.obs[cell_type_colname].iloc[0]
            adata_rep.obs_names = [f"{samp_name}_{cell}"]
            
            # Fill in metadata and number of cells
            adata_rep.obs[cell_type_colname] = cell
            adata_rep.obs[metacols] = adata_sub_ct.obs[metacols].iloc[0]
            adata_rep.obs["n_cells"] = n_cells
            
            ps_list.append(adata_rep)

    ps_adata = sc.concat(ps_list)
    ps_adata.obs["n_cells"] = ps_adata.obs["n_cells"].astype(int)
    ps_adata.obs['log2_n_cells'] = np.log2(ps_adata.obs['n_cells'])

    return ps_adata


In [2]:
import scanpy as sc
import numpy as np
import random

def make_pseudobulk3(adata, sample_colname, cell_type_colname, metacols, min_cells=40, max_cells=50, random_state=0):
    '''
    Creates an AnnData object with pseudo-bulk gene expression profiles 
    for the samples and cell types specified. Also adds the number of cells 
    used per pseudobulk as "n_cells" in .obs.

    Parameters:
        adata: AnnData
        sample_colname: str
        cell_type_colname: str
        metacols: list of str
        min_cells: int, minimum number of cells required to include a group
        max_cells: int, maximum number of cells to include per group (randomly sampled if exceeded)
        random_state: int, for reproducibility of random sampling

    Returns:
        AnnData with pseudobulked profiles and metadata.
    '''
    ps_list = []
    rng = np.random.default_rng(random_state)

    for s in adata.obs[sample_colname].unique():
        adata_sub = adata[adata.obs[sample_colname] == s]
        
        for ct in adata_sub.obs[cell_type_colname].unique():
            adata_sub_ct = adata_sub[adata_sub.obs[cell_type_colname] == ct]
            n_cells_total = adata_sub_ct.shape[0]

            # Skip if too few cells
            if n_cells_total < min_cells:
                continue

            # Downsample if too many
            if n_cells_total > max_cells:
                selected_idx = rng.choice(adata_sub_ct.shape[0], size=max_cells, replace=False)
                adata_sub_ct = adata_sub_ct[selected_idx]
            else:
                selected_idx = slice(None)  # Use all

            n_cells = adata_sub_ct.shape[0]

            adata_rep = sc.AnnData(X=adata_sub_ct.X.sum(axis=0),
                                   var=adata_sub_ct.var[[]])
            
            samp_name = adata_sub.obs[sample_colname].iloc[0]
            cell = adata_sub_ct.obs[cell_type_colname].iloc[0]
            adata_rep.obs_names = [f"{samp_name}_{cell}"]
            
            # Fill in metadata and number of cells
            adata_rep.obs[cell_type_colname] = cell
            adata_rep.obs[metacols] = adata_sub_ct.obs[metacols].iloc[0]
            adata_rep.obs["n_cells"] = n_cells
            
            ps_list.append(adata_rep)

    ps_adata = sc.concat(ps_list)
    ps_adata.obs["n_cells"] = ps_adata.obs["n_cells"].astype(int)
    ps_adata.obs['log2_n_cells'] = np.log2(ps_adata.obs['n_cells']).astype(float)

    return ps_adata


In [11]:
import pandas as pd
import scipy.sparse

def savePBdata2(adata, filename):
    # Convert counts to dense matrix if it's sparse
    if scipy.sparse.issparse(adata.X):
        counts = adata.X.toarray()
    else:
        counts = adata.X

    # Convert to DataFrame
    counts_df = pd.DataFrame(counts, index=adata.obs_names, columns=adata.var_names)

    # (Optional) Convert to integer
    counts_df = counts_df.astype(int, errors='ignore')

    # Save AnnData object
    adata.write(filename)
    
    # Save counts as CSV for inspection (optional)
    counts_df.to_csv(filename.replace(".h5ad", "_counts.csv"))

    print(f"Saved AnnData to {filename} and count matrix to CSV.")


In [8]:
ps_adata_macs = make_pseudobulk(adata, sample_colname = 'sample', cell_type_colname = 'Projection_CellType', metacols= ['assay', 'donor_id', 'disease', 'tissue', 'study', 'sex', 'age', 'uicc_stage', 'tumor_stage'])

/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.
  warnings.warn(msg, ImplicitModificationWarning)
/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.
  warnings.warn(msg, ImplicitModificationWarning)
/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.
  warnings.warn(msg, ImplicitModificationWarning)
/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.
  warnings.warn(msg, ImplicitModificationWarning)
/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use 

In [25]:
ps_adata_macs = make_pseudobulk2(adata, sample_colname = 'sample', cell_type_colname = 'Projection_CellType', metacols= ['assay', 'donor_id', 'disease', 'tissue', 'study', 'sex', 'age', 'uicc_stage', 'tumor_stage'])

/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.
  warnings.warn(msg, ImplicitModificationWarning)
/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.
  warnings.warn(msg, ImplicitModificationWarning)
/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.
  warnings.warn(msg, ImplicitModificationWarning)
/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.
  warnings.warn(msg, ImplicitModificationWarning)
/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use 

In [12]:
#Pseudo-bulking with random down sampling once. min_cells=40, max_cells=50
ps_adata_macs = make_pseudobulk3(adata, sample_colname = 'sample', cell_type_colname = 'Projection_CellType', metacols= ['assay', 'donor_id', 'disease', 'tissue', 'study', 'sex', 'age', 'uicc_stage', 'tumor_stage'])

/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.
  warnings.warn(msg, ImplicitModificationWarning)
/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.
  warnings.warn(msg, ImplicitModificationWarning)
/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.
  warnings.warn(msg, ImplicitModificationWarning)
/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.
  warnings.warn(msg, ImplicitModificationWarning)
/home/biolab/miniconda3/lib/python3.13/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use 

In [13]:
ps_adata_macs.obs.shape

(613, 12)

In [14]:
ps_adata_macs.obs

,Projection_CellType,assay,donor_id,disease,tissue,study,sex,age,uicc_stage,tumor_stage,n_cells,log2_n_cells
Adams_Kaminski_2020_001C_RTM_TAMs,RTM_TAMs,10x 3' v2,Adams_Kaminski_2020_001C,normal,lung,Adams_Kaminski_2020,male,22.0,non-cancer,non-cancer,50,5.643856
Adams_Kaminski_2020_001C_LA_TAMs,LA_TAMs,10x 3' v2,Adams_Kaminski_2020_001C,normal,lung,Adams_Kaminski_2020,male,22.0,non-cancer,non-cancer,50,5.643856
Adams_Kaminski_2020_001C_Int.Node.3,Int.Node.3,10x 3' v2,Adams_Kaminski_2020_001C,normal,lung,Adams_Kaminski_2020,male,22.0,non-cancer,non-cancer,45,5.491853
Adams_Kaminski_2020_002C_RTM_TAMs,RTM_TAMs,10x 3' v2,Adams_Kaminski_2020_002C,normal,lung,Adams_Kaminski_2020,female,25.0,non-cancer,non-cancer,50,5.643856
Adams_Kaminski_2020_002C_LA_TAMs,LA_TAMs,10x 3' v2,Adams_Kaminski_2020_002C,normal,lung,Adams_Kaminski_2020,female,25.0,non-cancer,non-cancer,50,5.643856
...,...,...,...,...,...,...,...,...,...,...,...,...
Leader_Merad_2021_414_RTM_TAMs,RTM_TAMs,10x 3' v2,Leader_Merad_2021_729,lung adenocarcinoma,lung,Leader_Merad_2021,female,64.0,II,early,50,5.643856
Leader_Merad_2021_414_Angio_TAMs,Angio_TAMs,10x 3' v2,Leader_Merad_2021_729,lung adenocarcinoma,lung,Leader_Merad_2021,female,64.0,II,early,50,5.643856
Leader_Merad_2021_414_IFN_TAMs,IFN_TAMs,10x 3' v2,Leader_Merad_2021_729,lung adenocarcinoma,lung,Leader_Merad_2021,female,64.0,II,early,50,5.643856
Leader_Merad_2021_414_Reg_TAMs,Reg_TAMs,10x 3' v2,Leader_Merad_2021_729,lung adenocarcinoma,lung,Leader_Merad_2021,female,64.0,II,early,50,5.643856


In [15]:
savePBdata2(ps_adata_macs, filename="../data/ps_adata_macs.h5ad")


Saved AnnData to ../data/ps_adata_macs.h5ad and count matrix to CSV.


In [16]:
with open('../data/Homo_sapiens.GRCh38.104.gtf') as f:
    gtf = list(f)

#prep the gtf file
gtf = [x for x in gtf if not x.startswith('#')]
gtf = [x for x in gtf if 'gene_id "' in x and 'gene_name "' in x]
gtf = list(map(lambda x: (x.split('gene_id "')[1].split('"')[0], x.split('gene_name "')[1].split('"')[0]), gtf))

In [17]:
ctyps = ps_adata_macs.obs['Projection_CellType'][~ps_adata_macs.obs['Projection_CellType'].isin(['Int.Node.3', 'Int.Node.4', 'Int.Node.5'])].unique()
ctyps

['RTM_TAMs', 'LA_TAMs', 'Reg_TAMs', 'IFN_TAMs', 'Angio_TAMs', 'Inflam_TAMs', 'Prolif_TAMs']
Categories (9, object): ['Angio_TAMs', 'IFN_TAMs', 'Inflam_TAMs', 'Int.Node.3', ..., 'LA_TAMs', 'Prolif_TAMs', 'RTM_TAMs', 'Reg_TAMs']

In [18]:
#sub_canc = ps_adata_macs[ps_adata_macs.obs['cell_type'] == 'malignant cell' ]
df = pd.DataFrame()

for c in ctyps:
    sub_c = ps_adata_macs[ps_adata_macs.obs['Projection_CellType'] == c ]
    sub_c.obs['Contrast'] = c
    pbs = [sub_c]
    print(c)
    sub_ct = ps_adata_macs[ps_adata_macs.obs['Projection_CellType'] != c ]
    sub_ct.obs['Contrast'] = 'others'
    pbs.append(sub_ct)
    pb = sc.concat(pbs)
    print(pb)
    counts= pd.DataFrame(pb.X, columns = pb.var_names)
    counts = counts.astype(int, errors='ignore')
    #To get rid off all the 0s in count matrix (to prevent problems)
    counts=counts+1
    dds = DeseqDataSet(counts = counts, metadata=pb.obs, design_factors = ["assay", "Contrast"])
    sc.pp.filter_genes(dds, min_cells = 10)
    dds.deseq2()
    stat_res = DeseqStats(dds, contrast=('Contrast', c, 'others'))
    stat_res.summary()
    de  = stat_res.results_df
    de = de.sort_values('log2FoldChange', ascending = False)
        
    #get rownames aka gene symbols
    row_names = de.index.tolist()
    #row_names
    #add rownames as a new column 
    de['gene_symbols'] = row_names
    gtf=dict(gtf)
    de['gene_name'] = de['gene_symbols'].map(gtf)
    de = de[(de['padj'] < 0.05) & (de['log2FoldChange'] > 1.0) ]
    de['SubType'] = c
    de['inContrastTo'] = 'others'
    df = pd.concat([df, de])
    #save deg list
    df.to_csv('../results/macs_PB-DEGs.csv', index=True)

/tmp/ipykernel_2280971/4100030739.py:6: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  sub_c.obs['Contrast'] = c
/tmp/ipykernel_2280971/4100030739.py:10: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  sub_ct.obs['Contrast'] = 'others'


RTM_TAMs
AnnData object with n_obs × n_vars = 613 × 17811
    obs: 'Projection_CellType', 'assay', 'donor_id', 'disease', 'tissue', 'study', 'sex', 'age', 'uicc_stage', 'tumor_stage', 'n_cells', 'log2_n_cells', 'Contrast'


/tmp/ipykernel_2280971/4100030739.py:18: DeprecationWarning: design_factors is deprecated and will soon be removed.Please consider providing a formulaic formula using the design argumentinstead.
  dds = DeseqDataSet(counts = counts, metadata=pb.obs, design_factors = ["assay", "Contrast"])
Fitting size factors...


Using None as control genes, passed at DeseqDataSet initialization


... done in 0.55 seconds.

/home/biolab/miniconda3/lib/python3.13/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=2280971) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/home/biolab/miniconda3/lib/python3.13/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=2280971) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/home/biolab/miniconda3/lib/python3.13/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=2280971) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/home/biolab/miniconda3/lib/python3.13/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=2280971) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/home/biolab/miniconda3/lib/python3.1

...

/tmp/ipykernel_2280971/4100030739.py:18: DeprecationWarning: design_factors is deprecated and will soon be removed.Please consider providing a formulaic formula using the design argumentinstead.
  dds = DeseqDataSet(counts = counts, metadata=pb.obs, design_factors = ["assay", "Contrast"])
Fitting size factors...


Using None as control genes, passed at DeseqDataSet initialization


... done in 0.55 seconds.

Fitting dispersions...
... done in 5.02 seconds.

Fitting dispersion trend curve...
/home/biolab/miniconda3/lib/python3.13/site-packages/pydeseq2/dds.py:804: UserWarning: The dispersion trend curve fitting did not converge. Switching to a mean-based dispersion trend.
  self._fit_parametric_dispersion_trend(vst)
... done in 0.27 seconds.

Fitting MAP dispersions...
... done in 5.04 seconds.

Fitting LFCs...
... done in 6.28 seconds.

Calculating cook's distance...
... done in 0.97 seconds.

Replacing 17 outlier genes.

Fitting dispersions...
... done in 0.03 seconds.

Fitting MAP dispersions...
... done in 0.05 seconds.

Fitting LFCs...
... done in 0.05 seconds.

Running Wald tests...


Log2 fold change & Wald test p-value: Contrast Prolif_TAMs vs others
                  baseMean  log2FoldChange     lfcSE      stat    pvalue  \
ENSG00000121410   8.899818       -0.503848  0.205633 -2.450224  0.014277   
ENSG00000268895   2.051674       -0.205029  0.271770 -0.754420  0.450597   
ENSG00000175899  42.494688       -1.719012  0.370766 -4.636382  0.000004   
ENSG00000245105   1.312772        0.103912  0.319890  0.324836  0.745305   
ENSG00000166535   1.299172       -0.051154  0.350714 -0.145856  0.884035   
...                    ...             ...       ...       ...       ...   
ENSG00000070476   4.463204       -0.608320  0.313775 -1.938715  0.052536   
ENSG00000203995   1.254236        0.047520  0.342024  0.138937  0.889500   
ENSG00000162378   8.232734       -0.141577  0.190036 -0.745001  0.456271   
ENSG00000159840  37.619983       -0.562453  0.171196 -3.285425  0.001018   
ENSG00000074755   8.015019       -0.667045  0.268499 -2.484347  0.012979   

                  

... done in 5.17 seconds.

